In [2]:
import pandas as pd
enrollment=pd.read_excel('Marshall_Course_Enrollment_1516_1617.xlsx')
cancelled=pd.read_excel('Cancelled_Courses_1516_1617.xlsx')
enrollment.head()

,Course,Course Prefix,Course Suffix,Department,First Begin Time,First Days,First End Time,First Instructor,First Instructor UID,First Room,...,Second Begin Time,Second Days,Second End Time,Second Instructor,Second Instructor UID,Second Room,Section,Session,Term,Title
0,ACCT-370,ACCT,370,ACCT,10:00:00,F,11:50:00,"Hopkins, Merle, W",3.783354e+09,SLH200,...,NaN,NaN,NaN,NaN,NaN,NaN,14029,1,20153,External Financial Reporting Issues
1,ACCT-370,ACCT,370,ACCT,08:00:00,MW,09:50:00,"Hopkins, Merle, W",3.783354e+09,ACC303,...,NaN,NaN,NaN,NaN,NaN,NaN,14025,1,20153,External Financial Reporting Issues
2,ACCT-370,ACCT,370,ACCT,10:00:00,MW,11:50:00,"Hopkins, Merle, W",3.783354e+09,ACC303,...,NaN,NaN,NaN,NaN,NaN,NaN,14026,1,20153,External Financial Reporting Issues
3,ACCT-370,ACCT,370,ACCT,12:00:00,MW,13:50:00,"Hopkins, Merle, W",3.783354e+09,ACC303,...,NaN,NaN,NaN,NaN,NaN,NaN,14027,1,20153,External Financial Reporting Issues
4,ACCT-371,ACCT,371,ACCT,10:00:00,F,11:50:00,NaN,NaN,SLH200,...,NaN,NaN,NaN,NaN,NaN,NaN,14044,1,20153,Introduction to Accounting Systems


Because there are many NAs in column 'Course', we need to combine 'Course Prefix' and 'Course Suffix' to get the full list of cancelled courses.

In [3]:
cancelled['full_course'] = cancelled['Course Prefix'].astype(str)+'-'+ cancelled['Course Suffix'].astype(str)
cancelled.head()

,Course,Course Prefix,Course Suffix,Department,First Begin Time,First Days,First End Time,First Instructor,First Instructor UID,First Room,...,Second Days,Second End Time,Second Instructor,Second Instructor UID,Second Room,Section,Session,Term,Title,full_course
0,ACCT-478,ACCT,478,ACCT,14:00:00,MW,15:50:00,"Kiddoo, Bob",7.682197e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,14160,1,20153,Accounting Systems Design,ACCT-478
1,ACCT-550T,ACCT,550T,ACCT,14:00:00,TH,16:50:00,"Mills, Patricia",7.334830e+09,JKP202,...,NaN,NaN,NaN,NaN,NaN,14218,431,20153,Tax Research and Professional Responsibilities,ACCT-550T
2,ACCT-561T,ACCT,561T,ACCT,18:30:00,H,21:30:00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,14238,1,20153,Income Tax of Corporations and Their Shareholders,ACCT-561T
3,ACCT-561T,ACCT,561T,ACCT,11:00:00,MW,12:00:00,"Keller, Joe",8.443357e+09,NaN,...,NaN,NaN,"Maxey, Shirley, Orechwa",8.813329e+09,NaN,14250,1,20153,Income Tax of Corporations and Their Shareholders,ACCT-561T
4,ACCT-573T,ACCT,573T,ACCT,17:30:00,W,18:30:00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,14265,378,20153,Federal Tax Procedure,ACCT-573T


In [4]:
# count the cancelled amount for each course by term
count_cancelled = cancelled.groupby(['Term','full_course']).size().reset_index(name = 'cancelled_amount')
count_cancelled.head()

,Term,full_course,cancelled_amount
0,20153,ACCT-370,4
1,20153,ACCT-371,4
2,20153,ACCT-372,4
3,20153,ACCT-373,4
4,20153,ACCT-374,3


In [5]:
count_enroll = enrollment.groupby(['Term','Course']).size().reset_index(name = 'enrolled_amount')
count_enroll.head()

,Term,Course,enrolled_amount
0,20153,ACCT-370,4
1,20153,ACCT-371,4
2,20153,ACCT-372,4
3,20153,ACCT-373,4
4,20153,ACCT-374,3


In [6]:
# merge two datasets to get the original plan for all courses
total =pd.merge(count_cancelled, count_enroll,  how='outer', 
                left_on=['Term','full_course'], right_on = ['Term','Course'])
total = total.fillna(0) #to fill out all the NAs to 0
total['cancelled_amount'] = total['cancelled_amount'].astype(int)
total['enrolled_amount'] = total['enrolled_amount'].astype(int)


In [7]:
# calculate the cancel rate for each course per semester
total['total_amount'] = total['cancelled_amount']+total['enrolled_amount']
total['cancel_rate'] = total['cancelled_amount']/total['total_amount']


In [12]:
#get the all the cancelled courses, filtering all the cancel_rate = 0
df1 = total[(total.cancel_rate != 0)]
df1.sort_values(['Term','cancel_rate'], ascending=[True,False])


,Term,full_course,cancelled_amount,Course,enrolled_amount,total_amount,cancel_rate
15,20153,ACCT-478,1,0,0,1,1.000000
35,20153,ACCT-590,1,0,0,1,1.000000
46,20153,BAEP-495,1,0,0,1,1.000000
47,20153,BAEP-497,1,0,0,1,1.000000
59,20153,BAEP-596,1,0,0,1,1.000000
60,20153,BAEP-597,1,0,0,1,1.000000
65,20153,BUAD-206A,1,0,0,1,1.000000
88,20153,BUCO-425,1,0,0,1,1.000000
91,20153,BUCO-504,1,0,0,1,1.000000
92,20153,BUCO-533,1,0,0,1,1.000000
